In [1]:
import sys
import numpy as np
import pandas as pd
import edlib
from bignmf.models.snmf.standard import StandardNmf
from scipy.spatial.distance import pdist, squareform
from sklearn.decomposition import NMF
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import math
import random

In [21]:
def read_file_ribo(file_name, reads_modifs_dict):
    with open(file_name, "r") as f:
        line = f.readline()
        index = 0
        while line != "":
            mod_clust = line.split("\t")
            name = mod_clust[0]
            modifs = mod_clust[1].split(" ")
            m = bin(int(''.join(map(str, modifs)), 2) << 1)
            reads_modifs_dict[name] = (m, index)
            index += 1
            line = f.readline()

In [46]:
def prepare_data(reads_modifs_dict, reads_name):
    reads_modifs_dict_subset = {}
    sub_set = list(reads_modifs_dict.keys())
    for i, read in enumerate(sub_set):
        name = "read_" + str(i)
        reads_name[read] = name
        reads_modifs_dict_subset[name] = reads_modifs_dict[read][0]
    reads_modifs_dict = reads_modifs_dict_subset.copy()
    return reads_modifs_dict

In [47]:
def similarity_function(r1, r2):
    intersection = str(bin(int(r1[2:], 2) & int(r2[2:], 2)))[2:].count("1")
    if intersection == 0:
        return 0
    union = str(bin(int(r1[2:], 2) | int(r2[2:], 2)))[2:].count("1")
    return float(intersection)/union

In [48]:
def similarity_jacc(r1, r2):
    r1 = np.array(r1)
    print(r1)
    r2 = np.array(r2)
    sum_r = np.add(r1, r2)
    union = np.count_nonzero(sum_r)
    inter = np.count_nonzero(sum_r == 2)
    return inter/union

In [49]:
def similarity_function_edlib(r1, r2):
    sim = 0
    try:
        align_result = edlib.align(r1, r2, mode="PW", task="path")
    except:
        return sim
    
    distance = align_result['editDistance']
    if distance > 0:
        sim = 1 - (1/distance)
    else:
        sim = 1.0
    return sim

In [50]:
def calculate_sim_matrix(data, reads_modifs_dict):
    i = 0
    for k1 in reads_modifs_dict.keys():
        i += 1
        r1 = reads_modifs_dict[k1]
        ind1 = int(k1[5:])
        for k2 in reads_modifs_dict.keys():
            r2 = reads_modifs_dict[k2]
            ind2 = int(k2[5:])
            if data[ind1][ind2] >= 0:
                continue
            if k1 == k2:
                sim = 1.0
            else:
                sim = similarity_function(r1, r2)
            data[ind1][ind2] = sim
            data[ind2][ind1] = sim

In [51]:
def NMF_iteration(num_clusters, data, names, clusters_freq, clusters_reads, inertia_array, threshold):
    iteration = 0
    razvrstani = []

    df = pd.DataFrame(data, columns=names, index=names)
    
    while len(razvrstani) < len(data[0]):
        iteration += 1
        model = NMF(n_components=num_clusters, init='random', random_state=0, tol=1e-3, max_iter=2000)
        W = model.fit_transform(df)
        H = model.components_
        err = model.reconstruction_err_
    
        nerazvrstani = []
        novi_razvrstani = []
        klasteri_novi_razvrstani = []
        for i, index in enumerate(df.index.values.tolist()):
            max_index = -1
            max_value = -1.
            for j in range(len(H)):
                if H[j][i] > max_value:
                    max_index = j
                    max_value = H[j][i]
        
            for_cluster = True
            for j in range(len(H)):
                if j == max_index:
                    continue
                if max_value - H[j][i] < threshold:
                    for_cluster = False
                    break
            if for_cluster:
                clusters_freq[max_index] += 1
                cluster_list = clusters_reads[max_index]
                cluster_list.append(index)
                clusters_reads[max_index] = cluster_list
                razvrstani.append(index)
                novi_razvrstani.append(index)
                klasteri_novi_razvrstani.append(max_index)
            else:
                nerazvrstani.append(index)
        if len(novi_razvrstani) == 0:
            break
        
        for i,read in enumerate(novi_razvrstani):
            centroid = [item[klasteri_novi_razvrstani[i]] for item in W]
            inertia = np.linalg.norm(np.array(df[read].values) - np.array(centroid))
            inertia_array.append(inertia)
        
        df = df.drop(novi_razvrstani)
        df = df.drop(novi_razvrstani, axis=1)
    
        print("Number of not clustered reads: " + str(len(nerazvrstani)))
    
    for i, index in enumerate(df.index.values.tolist()):
        max_index = -1
        max_value = -1.
        for j in range(len(H)):
            if H[j][i] > max_value:
                max_index = j
                max_value = H[j][i]
        clusters_freq[max_index] += 1
        cluster_list = clusters_reads[max_index]
        cluster_list.append(index)
        clusters_reads[max_index] = cluster_list
        razvrstani.append(index)
        
        centroid = [item[max_index] for item in W]
        inertia = np.linalg.norm(np.array(df[index].values) - np.array(centroid))
        inertia_array.append(inertia)

    print(clusters_freq)

In [52]:
def save_to_file(file_name, clusters_reads, reads_name):
    with open(file_name, 'w') as f:
        for cluster, reads in clusters_reads.items():
            for read in reads:
                for r, name in reads_name.items():
                    if name == read:
                        read_name = r
                        break
                line = read_name + " : " + str(cluster) + "\n"
                f.write(line)

In [ ]:
reads_modifs_dict = {}
read_file_ribo("MPRS21_all.txt", reads_modifs_dict)
reads_name = {}
reads_modifs_dict = prepare_data(reads_modifs_dict, reads_name)
data = np.full((len(list(reads_modifs_dict.keys())), len(list(reads_modifs_dict.keys()))), -1.0)
calculate_sim_matrix(data, reads_modifs_dict)

clusters_freq = {0:0, 1:0}
clusters_reads = {0:[], 1:[]}
inertia_array = []

threshold = 0.1
NMF_algorithm(2, data, list(reads_modifs_dict.keys()), clusters_freq, clusters_reads, inertia_array, threshold)
inertia_array = np.array(inertia_array)

save_to_file("MPRS21_clusters_NMF.txt", clusters_reads, reads_name)